In [1]:
import pandas as pd
from pathlib import Path
from ast import literal_eval
import json
from collections import defaultdict

pd.set_option('display.max_columns', None)

In [2]:
DATA_DIR = Path('../data/')

In [3]:
address_data = pd.read_csv(DATA_DIR / 'data-60562-2024-05-24.csv', sep=';', low_memory=False)
column_description = address_data.iloc[0]
address_data = address_data.iloc[1:].reset_index(drop=True)

In [4]:
column_description

global_id                                                      global_id
OBJ_TYPE                                           Тип объекта адресации
OnTerritoryOfMoscow                                 На территории Москвы
ADDRESS                Полное юридическое написание адреса или описан...
UNOM                   Учётный номер объекта адресации в БД БТИ (кром...
P1                                                            Субъект РФ
P3                                                             Поселение
P4                                                                 Город
P5                                                   Муниципальный округ
P6                                                      Населённый пункт
P7                     Наименование элемента планировочной структуры ...
P90                              Дополнительный адресообразующий элемент
SIMPLE_ADDRESS         Упрощённое написание адреса или описание место...
P91                    Уточнение дополнительного ад

In [5]:
address_data.OBJ_TYPE.value_counts()

OBJ_TYPE
Здание                                 286842
Земельный участок                      197208
Сооружение                               6974
объект незавершенного строительства       331
Name: count, dtype: int64

In [5]:
filtered_data = address_data[
    (address_data.OnTerritoryOfMoscow == 'да') & 
    (address_data.ADM_AREA.isin((
        'Центральный административный округ',
        'Западный административный округ',
        'Восточный административный округ',
        'Северный административный округ',
        'Южный административный округ',
        'Юго-Восточный административный округ',
        'Северо-Восточный административный округ',
        'Юго-Западный административный округ',
        'Северо-Западный административный округ'
    ))) &
    (address_data.P5.notna()) &
    (address_data.geodata_center.notna()) &
    (address_data.VID == 'Адрес') &
    (address_data.P6.isna()) &
    (address_data.OBJ_TYPE == 'Здание') &
    (address_data.L1_TYPE == 'дом') &
    (address_data.L3_TYPE != 'сооружение')
]

display(filtered_data.shape)

(126996, 45)

In [14]:
filtered_data_target_cols = filtered_data[['global_id', 'DISTRICT', 'SIMPLE_ADDRESS', 'ADM_AREA', 'geodata_center']].copy()
filtered_data_target_cols.geodata_center = filtered_data_target_cols.geodata_center.apply(lambda x: x.split('=')[1].split(', type')[0])
filtered_data_target_cols.reset_index(drop=True, inplace=True)

In [16]:
filtered_data_target_cols.to_json('../data/mos_ru_filtered_data.json')

In [24]:
adm_to_district = defaultdict(set)
for row in filtered_data.itertuples():
    adm_to_district[row.ADM_AREA].add(row.DISTRICT)
adm_to_district = {adm : list(district) for adm, district in adm_to_district.items()}

In [25]:
s = 0
for adm, district in adm_to_district.items():
    print(f'{adm} {len(district)}')
    s += len(district)
s

Восточный административный округ 16
Центральный административный округ 10
Северный административный округ 16
Юго-Восточный административный округ 12
Южный административный округ 16
Западный административный округ 13
Северо-Восточный административный округ 17
Юго-Западный административный округ 12
Северо-Западный административный округ 8


120

In [30]:
with open('adm_to_district.json', 'w') as fh:
    fh.write(json.dumps(adm_to_district, ensure_ascii=False))

In [218]:
sampled_data = filtered_data[['global_id', 'ADDRESS', 'SIMPLE_ADDRESS', 'ADM_AREA', 'DISTRICT', 'geodata_center']].iloc[:1000]
sampled_data.geodata_center = sampled_data.geodata_center.apply(lambda x: x.split('=')[1].split(', type')[0])

In [225]:
sampled_data.reset_index

,global_id,ADDRESS,SIMPLE_ADDRESS,ADM_AREA,DISTRICT,geodata_center
0,645622141,"Российская Федерация, город Москва, внутригоро...","Косинская улица, дом 26А",Восточный административный округ,муниципальный округ Вешняки,"[37.828189394, 55.717482785]"
1,645622142,"Российская Федерация, город Москва, внутригоро...","Гороховский переулок, дом 21",Центральный административный округ,муниципальный округ Басманный,"[37.668013072, 55.766228272]"
2,645622174,"Российская Федерация, город Москва, внутригоро...","Ленинградский проспект, дом 73А, строение 3",Северный административный округ,муниципальный округ Сокол,"[37.513482336, 55.803579031]"
3,645622205,"Российская Федерация, город Москва, внутригоро...","улица Вострухина, дом 5А",Юго-Восточный административный округ,муниципальный округ Рязанский,"[37.797663794, 55.720046086]"
4,645622216,"Российская Федерация, город Москва, внутригоро...","улица Космонавта Волкова, дом 10, строение 1",Северный административный округ,муниципальный округ Войковский,"[37.514882646, 55.815345188]"
...,...,...,...,...,...,...
1047,645641524,"Российская Федерация, город Москва, внутригоро...","Шипиловская улица, дом 64, корпус 2",Южный административный округ,муниципальный округ Зябликово,"[37.753665096, 55.622334284]"
1048,645641535,"Российская Федерация, город Москва, внутригоро...","Железнодорожный проезд, дом 9, строение 2",Южный административный округ,муниципальный округ Даниловский,"[37.636773865, 55.710755518]"
1049,645641556,"Российская Федерация, город Москва, внутригоро...","Перекопская улица, дом 7, корпус 1",Юго-Западный административный округ,муниципальный округ Зюзино,"[37.58148713, 55.658654811]"
1050,645641577,"Российская Федерация, город Москва, внутригоро...","Миллионная улица, дом 11, корпус 2",Восточный административный округ,муниципальный округ Богородское,"[37.69607124, 55.817376717]"


In [224]:
sampled_data.to_json('sampled_addresses.json', orient='records', force_ascii=False)

In [188]:
filtered_data.N_FIAS.value_counts().head(20)

N_FIAS
FE67EAAA-1596-45EA-BBC5-FC852A39B72A    4
5FA16943-7C54-48D0-9642-EC898DCD9923    4
24AD9ED8-1EA8-4054-93D2-494F657BB6B6    4
E10B02FE-B75A-40C9-9B65-05E7F2F967B7    4
FE332D8D-D830-43DE-894E-96779A9C17F8    3
D76D007D-4A5E-4E5F-ABF2-A358650CA764    3
0CFD6E10-55A9-48BF-9A19-9D81919642D5    3
40EAF155-0C6E-4051-8D85-A796B491FB16    3
CF98E1C7-59D3-42EC-9FE6-A92924065FAB    3
1A272A1E-1A0B-462F-9F33-2EF88371749A    3
871C5274-456B-449D-BB7B-5A4175B2F2CA    3
9077794C-2C37-428F-868F-F10AE039D1A0    2
57A3EF2F-88A3-47DE-A27B-D966EEF53148    2
2F0E62C5-C956-4098-AE68-08E7BC354E40    2
C5FFE014-AA17-4BE0-802C-FDD16491EDA2    2
2B301CE5-05A2-4A35-85F0-0898DE3F382D    2
DB551F48-BA9B-4D3A-9980-141D9CCDAC5F    2
FD550647-D398-4460-BD20-D6297E385A46    2
4F212FE6-8B87-4C78-9306-6B183479641E    2
F3A0B9A5-146A-4231-B281-E309F70613A3    2
Name: count, dtype: int64

In [37]:
filtered_data[filtered_data.SIMPLE_ADDRESS.apply(lambda x: 'Нагатинская набережная, дом 48/2' in x)]

,global_id,OBJ_TYPE,OnTerritoryOfMoscow,ADDRESS,UNOM,P1,P3,P4,P5,P6,P7,P90,SIMPLE_ADDRESS,P91,L1_TYPE,L1_VALUE,L2_TYPE,P0,L2_VALUE,L3_TYPE,P2,L3_VALUE,L4_TYPE,L4_VALUE,L5_TYPE,L5_VALUE,ADM_AREA,DISTRICT,NREG,DREG,N_FIAS,D_FIAS,KAD_N,KAD_ZU,KLADR,TDOC,NDOC,DDOC,ADR_TYPE,VID,SOSTAD,STATUS,geoData,geodata_center,Unnamed: 44
23956,646093616,Здание,да,"город Москва, Нагатинская набережная, дом 48/2...",3807723,город Москва,NaN,NaN,муниципальный округ Нагатинский Затон,NaN,Нагатинская набережная,NaN,"Нагатинская набережная, дом 48/2, строение 3",NaN,дом,48/2,NaN,NaN,NaN,строение,NaN,3,NaN,NaN,NaN,NaN,Южный административный округ,муниципальный округ Нагатинский Затон,5013310,12.01.2006,9D9D23E4-15E4-45A3-B52D-E263EB995841,23.05.2013,nested data,nested data,77000000000194300,Распоряжение префектуры АО города Москвы,01-41-3840,29.12.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.680123584, 55.687235989], [...","{coordinates=[37.680073722, 55.687194998], typ...",NaN
68916,647389699,Здание,да,"Российская Федерация, город Москва, внутригоро...",3803595,город Москва,NaN,NaN,муниципальный округ Нагатинский Затон,NaN,Нагатинская набережная,NaN,"Нагатинская набережная, дом 48/2, строение 2",NaN,дом,48/2,NaN,Российская Федерация,NaN,строение,внутригородская территория муниципальный округ...,2,NaN,NaN,NaN,NaN,Южный административный округ,муниципальный округ Нагатинский Затон,5010379,25.10.2004,9110D9F1-A5F3-4397-8DB0-2A467F625BB4,24.11.2011,nested data,nested data,77000000000194300,Распоряжение префектуры АО города Москвы,01-41-2080,23.08.2004,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.680704568, 55.687334947], [...","{coordinates=[37.680601813, 55.687220458], typ...",NaN
114185,648954930,Здание,да,"Российская Федерация, город Москва, внутригоро...",15495,город Москва,NaN,NaN,муниципальный округ Нагатинский Затон,NaN,Нагатинская набережная,NaN,"Нагатинская набережная, дом 48/2",NaN,дом,48/2,NaN,Российская Федерация,NaN,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Южный административный округ,муниципальный округ Нагатинский Затон,5010381,25.10.2004,A5F2DB00-9663-4475-9D4E-8821F47FE003,27.02.2012,nested data,nested data,77000000000194300,Распоряжение префектуры АО города Москвы,01-41-2080,23.08.2004,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.6812484, 55.688154853], [37...","{coordinates=[37.679221432, 55.687336217], typ...",NaN


In [195]:
filtered_data[filtered_data.N_FIAS == 'FE67EAAA-1596-45EA-BBC5-FC852A39B72A']

,global_id,OBJ_TYPE,OnTerritoryOfMoscow,ADDRESS,UNOM,P1,P3,P4,P5,P6,P7,P90,SIMPLE_ADDRESS,P91,L1_TYPE,L1_VALUE,L2_TYPE,P0,L2_VALUE,L3_TYPE,P2,L3_VALUE,L4_TYPE,L4_VALUE,L5_TYPE,L5_VALUE,ADM_AREA,DISTRICT,NREG,DREG,N_FIAS,D_FIAS,KAD_N,KAD_ZU,KLADR,TDOC,NDOC,DDOC,ADR_TYPE,VID,SOSTAD,STATUS,geoData,geodata_center,Unnamed: 44
32070,646293254,Здание,да,"Российская Федерация, город Москва, внутригоро...",9329,город Москва,NaN,NaN,муниципальный округ Ясенево,NaN,проезд Карамзина,NaN,"проезд Карамзина, дом 1, корпус 3",NaN,дом,1,корпус,Российская Федерация,3,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Юго-Западный административный округ,муниципальный округ Ясенево,6007226,10.10.2005,FE67EAAA-1596-45EA-BBC5-FC852A39B72A,27.02.2012,nested data,nested data,77000000000143400,Распоряжение префектуры АО города Москвы,1988-РП,22.09.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[[37.538679043, 55.594591289], ...","{coordinates=[37.53898915, 55.594467381], type...",NaN
41008,646540254,Здание,да,"проезд Карамзина, дом 1, корпус 3",9327,город Москва,NaN,NaN,муниципальный округ Ясенево,NaN,проезд Карамзина,NaN,"проезд Карамзина, дом 1, корпус 3",NaN,дом,1,корпус,Российская Федерация,3,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Юго-Западный административный округ,муниципальный округ Ясенево,6007226,10.10.2005,FE67EAAA-1596-45EA-BBC5-FC852A39B72A,27.02.2012,nested data,nested data,77000000000143400,Распоряжение префектуры АО города Москвы,1988-РП,22.09.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.539306065, 55.594119088], [...","{coordinates=[37.539141056, 55.59402347], type...",NaN
52969,646897166,Здание,да,"Российская Федерация, город Москва, внутригоро...",9326,город Москва,NaN,NaN,муниципальный округ Ясенево,NaN,проезд Карамзина,NaN,"проезд Карамзина, дом 1, корпус 3",NaN,дом,1,корпус,Российская Федерация,3,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Юго-Западный административный округ,муниципальный округ Ясенево,6007226,10.10.2005,FE67EAAA-1596-45EA-BBC5-FC852A39B72A,27.02.2012,nested data,nested data,77000000000143400,Распоряжение префектуры АО города Москвы,1988-РП,22.09.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.537842754, 55.593879731], [...","{coordinates=[37.538217385, 55.5937466], type=...",NaN
117615,649027431,Здание,да,"проезд Карамзина, дом 1, корпус 3",9328,город Москва,NaN,NaN,муниципальный округ Ясенево,NaN,проезд Карамзина,NaN,"проезд Карамзина, дом 1, корпус 3",NaN,дом,1,корпус,Российская Федерация,3,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Юго-Западный административный округ,муниципальный округ Ясенево,6007226,10.10.2005,FE67EAAA-1596-45EA-BBC5-FC852A39B72A,27.02.2012,nested data,nested data,77000000000143400,Распоряжение префектуры АО города Москвы,1988-РП,22.09.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.535652226, 55.594353378], [...","{coordinates=[37.53630682, 55.594154426], type...",NaN
